# Data Preprocessing Notebook

This notebook shows the preprocessing steps taken to create a 2 Mio Spotify songs dataset

#### Datasets

All Datasets can be found in the data folder

- https://www.kaggle.com/datasets/amitanshjoshi/spotify-1million-tracks (original_spotify_data/1mio_dataset)
- https://www.kaggle.com/datasets/tonygordonjr/spotify-dataset-2023?select=spotify-albums_data_2023.csv (original_spotify_data/2023_dataset)
- https://www.kaggle.com/datasets/yamaerenay/spotify-dataset-19212020-600k-tracks?select=tracks.csv (original_spotify_data/600k_dataset)

# Loading all Datasets

## 1 Million Songs Dataset
https://www.kaggle.com/datasets/amitanshjoshi/spotify-1million-tracks

In [13]:
from custom_utils import load_and_concatenate_parquet_files
one_mio_dataset = load_and_concatenate_parquet_files("data/original_spotify_data/1mio_dataset")
print(one_mio_dataset.shape)
display(one_mio_dataset.head())
print(one_mio_dataset.isna().sum())
print(one_mio_dataset.info())

(1159764, 20)


,Unnamed: 0,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0,Jason Mraz,I Won't Give Up,53QF56cjZA9RTuuMZDrSA6,68,2012,acoustic,0.483,0.303,4,-10.058,1,0.0429,0.6940,0.000000,0.1150,0.139,133.406,240166,3
1,1,Jason Mraz,93 Million Miles,1s8tP3jP4GZcyHDsjvw218,50,2012,acoustic,0.572,0.454,3,-10.286,1,0.0258,0.4770,0.000014,0.0974,0.515,140.182,216387,4
2,2,Joshua Hyslop,Do Not Let Me Go,7BRCa8MPiyuvr2VU3O9W0F,57,2012,acoustic,0.409,0.234,3,-13.711,1,0.0323,0.3380,0.000050,0.0895,0.145,139.832,158960,4
3,3,Boyce Avenue,Fast Car,63wsZUhUZLlh1OsyrZq7sz,58,2012,acoustic,0.392,0.251,10,-9.845,1,0.0363,0.8070,0.000000,0.0797,0.508,204.961,304293,4
4,4,Andrew Belle,Sky's Still Blue,6nXIYClvJAfi6ujLiKqEq8,54,2012,acoustic,0.430,0.791,6,-5.419,0,0.0302,0.0726,0.019300,0.1100,0.217,171.864,244320,4


Unnamed: 0           0
artist_name         15
track_name           1
track_id             0
popularity           0
year                 0
genre                0
danceability         0
energy               0
key                  0
loudness             0
mode                 0
speechiness          0
acousticness         0
instrumentalness     0
liveness             0
valence              0
tempo                0
duration_ms          0
time_signature       0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1159764 entries, 0 to 1159763
Data columns (total 20 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   Unnamed: 0        1159764 non-null  int64  
 1   artist_name       1159749 non-null  object 
 2   track_name        1159763 non-null  object 
 3   track_id          1159764 non-null  object 
 4   popularity        1159764 non-null  int64  
 5   year              1159764 non-null  int64  
 6   genre            

In [14]:
one_mio_dataset = one_mio_dataset.drop(columns=["Unnamed: 0"])

In [17]:
one_mio_dataset = one_mio_dataset.dropna()
print(one_mio_dataset.isna().sum())
print(one_mio_dataset.shape)

artist_name         0
track_name          0
track_id            0
popularity          0
year                0
genre               0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
duration_ms         0
time_signature      0
dtype: int64
(1159748, 19)


## 600k Tracks Dataset
https://www.kaggle.com/datasets/yamaerenay/spotify-dataset-19212020-600k-tracks?select=tracks.csv

In [18]:
from custom_utils import load_and_concatenate_parquet_files
half_mio_dataset = load_and_concatenate_parquet_files("data/original_spotify_data/600k_dataset")
print(half_mio_dataset.shape)
display(half_mio_dataset.head())
print(half_mio_dataset.isna().sum())
print(half_mio_dataset.info())

(586672, 20)


,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4


id                   0
name                71
popularity           0
duration_ms          0
explicit             0
artists              0
id_artists           0
release_date         0
danceability         0
energy               0
key                  0
loudness             0
mode                 0
speechiness          0
acousticness         0
instrumentalness     0
liveness             0
valence              0
tempo                0
time_signature       0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 586672 entries, 0 to 586671
Data columns (total 20 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                586672 non-null  object 
 1   name              586601 non-null  object 
 2   popularity        586672 non-null  int64  
 3   duration_ms       586672 non-null  int64  
 4   explicit          586672 non-null  int64  
 5   artists           586672 non-null  object 
 6   id_artists        586672 no

In [19]:
half_mio_dataset = half_mio_dataset.dropna()
print(half_mio_dataset.isna().sum())
print(half_mio_dataset.shape)

id                  0
name                0
popularity          0
duration_ms         0
explicit            0
artists             0
id_artists          0
release_date        0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
dtype: int64
(586601, 20)


In [27]:
display(one_mio_dataset.head(1))
display(half_mio_dataset.head(10))

,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Jason Mraz,I Won't Give Up,53QF56cjZA9RTuuMZDrSA6,68,2012,acoustic,0.483,0.303,4,-10.058,1,0.0429,0.694,0.0,0.115,0.139,133.406,240166,3


,track_id,track_name,popularity,duration_ms,artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,['Uli'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.744000,0.1510,0.1270,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,['Fernando Pessoa'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.000000,0.1480,0.6550,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,['Ignacio Corsini'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.021800,0.2120,0.4570,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,['Ignacio Corsini'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.918000,0.1040,0.3970,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,['Dick Haymes'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.130000,0.3110,0.1960,103.220,4
5,0BRXJHRNGQ3W4v9frnSfhu,Ave Maria,0,178933,['Dick Haymes'],1922,0.227,0.2610,5,-12.343,1,0.0382,0.994,0.247000,0.0977,0.0539,118.891,4
6,0Dd9ImXtAtGwsmsAD69KZT,La Butte Rouge,0,134467,['Francis Marty'],1922,0.510,0.3550,4,-12.833,1,0.1240,0.965,0.000000,0.1550,0.7270,85.754,5
7,0IA0Hju8CAgYfV1hwhidBH,La Java,0,161427,['Mistinguett'],1922,0.563,0.1840,4,-13.757,1,0.0512,0.993,0.000016,0.3250,0.6540,133.088,3
8,0IgI1UCz84pYeVetnl1lGP,Old Fashioned Girl,0,310073,['Greg Fieler'],1922,0.488,0.4750,0,-16.222,0,0.0399,0.620,0.006450,0.1070,0.5440,139.952,4
9,0JV4iqw2lSKJaHBQZ0e5zK,Martín Fierro - Remasterizado,0,181173,['Ignacio Corsini'],1922-03-29,0.548,0.0391,6,-23.228,1,0.1530,0.996,0.933000,0.1480,0.6120,75.595,3


In [24]:
half_mio_dataset = half_mio_dataset.rename(columns={"id": "track_id", "name": "track_name"})
display(half_mio_dataset.head(1))

,track_id,track_name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.445,0,-13.338,1,0.451,0.674,0.744,0.151,0.127,104.851,3


In [25]:
half_mio_dataset = half_mio_dataset.drop(columns=["explicit", "id_artists"])
display(half_mio_dataset.head(1))

,track_id,track_name,popularity,duration_ms,artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,['Uli'],1922-02-22,0.645,0.445,0,-13.338,1,0.451,0.674,0.744,0.151,0.127,104.851,3


In [33]:
import numpy as np
import re
def extract_year(date_str):
    date_str = str(date_str).strip()
    match = re.match(r'^(\d{4})', date_str)
    if match:
        return int(match.group(1))
    return np.nan
    
half_mio_dataset['year'] = half_mio_dataset['release_date'].apply(extract_year)
display(half_mio_dataset[["year", "release_date"]].head(5))
half_mio_dataset = half_mio_dataset.drop(columns=["release_date"])

,year,release_date
0,1922,1922-02-22
1,1922,1922-06-01
2,1922,1922-03-21
3,1922,1922-03-21
4,1922,1922


In [34]:
import pandas as pd
import numpy as np
import re
def extract_first_artist(artists_str):

    clean_str = re.sub(r'[\[\]\']', '', artists_str).strip()
    artists_list = [name.strip() for name in clean_str.split(',')]
    return artists_list[0] if artists_list else np.nan

half_mio_dataset['artist_name'] = half_mio_dataset['artists'].apply(extract_first_artist)
display(half_mio_dataset[["artist_name", "artists"]].head(5))
half_mio_dataset = half_mio_dataset.drop(columns=["artists"])

,artist_name,artists
0,Uli,['Uli']
1,Fernando Pessoa,['Fernando Pessoa']
2,Ignacio Corsini,['Ignacio Corsini']
3,Ignacio Corsini,['Ignacio Corsini']
4,Dick Haymes,['Dick Haymes']


In [37]:
half_mio_dataset["genre"] = None
half_mio_dataset = half_mio_dataset[one_mio_dataset.columns]

In [38]:
display(half_mio_dataset.head(1))
display(one_mio_dataset.head(1))

,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Uli,Carve,35iwgR4jXetI318WEWsa1Q,6,1922,None,0.645,0.445,0,-13.338,1,0.451,0.674,0.744,0.151,0.127,104.851,126903,3


,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Jason Mraz,I Won't Give Up,53QF56cjZA9RTuuMZDrSA6,68,2012,acoustic,0.483,0.303,4,-10.058,1,0.0429,0.694,0.0,0.115,0.139,133.406,240166,3


## 2023 Dataset
https://www.kaggle.com/datasets/tonygordonjr/spotify-dataset-2023?select=spotify-albums_data_2023.csv

In [56]:
from custom_utils import load_and_concatenate_parquet_files
newest_dataset = load_and_concatenate_parquet_files("data/original_spotify_data/2023_dataset")
print(newest_dataset.shape)
display(newest_dataset.head())
print(newest_dataset.isna().sum())
print(newest_dataset.info())

(375141, 49)


,album_id,album_name,album_popularity,album_type,artists,artist_0,artist_1,artist_2,artist_3,artist_4,...,time_signature,track_href,type,uri,valence,explicit,track_popularity,release_year,release_month,rn
0,4zH8bbEjLe0Gdbv2FVLjnK,Son Of Tabu,1,album,['AB'],AB,None,None,None,None,...,4.0,https://api.spotify.com/v1/tracks/5JucnTkqh8uC...,audio_features,spotify:track:5JucnTkqh8uCZGggL1xxvv,0.382,True,0.0,2023.0,April,1
1,4zH8bbEjLe0Gdbv2FVLjnK,Son Of Tabu,1,album,"['AB', '22nd Jim']",AB,22nd Jim,None,None,None,...,4.0,https://api.spotify.com/v1/tracks/4VWrKNG3ssyZ...,audio_features,spotify:track:4VWrKNG3ssyZeboTgQ7Kj1,0.193,True,1.0,2023.0,April,1
2,4zH8bbEjLe0Gdbv2FVLjnK,Son Of Tabu,1,album,"['AB', 'Veeze']",AB,Veeze,None,None,None,...,4.0,https://api.spotify.com/v1/tracks/6JsVrTLJPSOS...,audio_features,spotify:track:6JsVrTLJPSOSpgJSnZyfSK,0.835,True,1.0,2023.0,April,1
3,4zH8bbEjLe0Gdbv2FVLjnK,Son Of Tabu,1,album,['AB'],AB,None,None,None,None,...,4.0,https://api.spotify.com/v1/tracks/7pZpw1xJWIav...,audio_features,spotify:track:7pZpw1xJWIavCUGqXPPriP,0.252,True,0.0,2023.0,April,1
4,4zH8bbEjLe0Gdbv2FVLjnK,Son Of Tabu,1,album,['AB'],AB,None,None,None,None,...,4.0,https://api.spotify.com/v1/tracks/0ylqwc3gcJdJ...,audio_features,spotify:track:0ylqwc3gcJdJotKd4SfMiu,0.380,True,0.0,2023.0,April,1


album_id                  0
album_name                1
album_popularity          0
album_type                0
artists                   0
artist_0                  2
artist_1             288062
artist_2             336841
artist_3             361318
artist_4             369054
artist_id                 0
duration_sec              0
label                    52
release_date             22
total_tracks              0
track_id                  0
track_name                5
track_number              0
artist_genres             0
artist_popularity         0
followers                 0
name                      0
genre_0              165278
genre_1              255655
genre_2              297922
genre_3              326531
genre_4              343824
acousticness            777
analysis_url            777
danceability            777
duration_ms             777
energy                  777
instrumentalness        777
key                     777
liveness                777
loudness            

In [57]:
newest_dataset = newest_dataset[["artist_0", "track_name", "track_id", "release_year", "track_popularity", "danceability", "energy", "key", "loudness", "mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "duration_ms", "time_signature", "genre_0"]]
display(newest_dataset.head(1))

,artist_0,track_name,track_id,release_year,track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,genre_0
0,AB,Intro,5JucnTkqh8uCZGggL1xxvv,2023.0,0.0,0.622,0.554,5.0,-9.505,0.0,0.414,0.0305,0.000001,0.392,0.382,72.546,92064.0,4.0,None


In [58]:
newest_dataset = newest_dataset.rename(columns={"artist_0": "artist_name", "release_year": "year", "genre_0": "genre", "track_popularity": "popularity"})
display(newest_dataset.head(1))

,artist_name,track_name,track_id,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,genre
0,AB,Intro,5JucnTkqh8uCZGggL1xxvv,2023.0,0.0,0.622,0.554,5.0,-9.505,0.0,0.414,0.0305,0.000001,0.392,0.382,72.546,92064.0,4.0,None


In [59]:
print(newest_dataset["genre"].value_counts())

genre
karaoke                     15344
dance pop                    4496
classical                    3848
album rock                   3707
baroque                      3421
                            ...  
jovem guarda                    1
cornwall indie                  1
australian reggae fusion        1
tamil pop                       1
argentine reggae                1
Name: count, Length: 3008, dtype: int64


In [60]:
newest_dataset = newest_dataset.drop(columns=["genre"])

In [61]:
print(newest_dataset.isna().sum())
newest_dataset = newest_dataset.dropna()

artist_name           2
track_name            5
track_id              0
year                 22
popularity           10
danceability        777
energy              777
key                 777
loudness            777
mode                777
speechiness         777
acousticness        777
instrumentalness    777
liveness            777
valence             777
tempo               777
duration_ms         777
time_signature      777
dtype: int64


In [72]:
newest_dataset["genre"] = None
newest_dataset["year"] = newest_dataset["year"].astype(int)
newest_dataset["time_signature"] = newest_dataset["time_signature"].astype(int)
newest_dataset["duration_ms"] = newest_dataset["duration_ms"].astype(int)
newest_dataset["key"] = newest_dataset["key"].astype(int)
newest_dataset["mode"] = newest_dataset["mode"].astype(int)
newest_dataset["popularity"] = newest_dataset["popularity"].astype(int)

In [73]:
newest_dataset = newest_dataset[one_mio_dataset.columns]
display(newest_dataset.head(1))
display(one_mio_dataset.head(1))
display(half_mio_dataset.head(1))

,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,AB,Intro,5JucnTkqh8uCZGggL1xxvv,0,2023,None,0.622,0.554,5,-9.505,0,0.414,0.0305,0.000001,0.392,0.382,72.546,92064,4


,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Jason Mraz,I Won't Give Up,53QF56cjZA9RTuuMZDrSA6,68,2012,acoustic,0.483,0.303,4,-10.058,1,0.0429,0.694,0.0,0.115,0.139,133.406,240166,3


,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Uli,Carve,35iwgR4jXetI318WEWsa1Q,6,1922,None,0.645,0.445,0,-13.338,1,0.451,0.674,0.744,0.151,0.127,104.851,126903,3


## Combining all datasets

In [74]:
combined_dataset = pd.concat([one_mio_dataset, half_mio_dataset, newest_dataset], ignore_index=True)
combined_dataset = combined_dataset.reset_index(drop=True)
display(combined_dataset.head(5))
print(combined_dataset.shape)
print(combined_dataset.isna().sum())
print(combined_dataset.info())

,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Jason Mraz,I Won't Give Up,53QF56cjZA9RTuuMZDrSA6,68,2012,acoustic,0.483,0.303,4,-10.058,1,0.0429,0.6940,0.000000,0.1150,0.139,133.406,240166,3
1,Jason Mraz,93 Million Miles,1s8tP3jP4GZcyHDsjvw218,50,2012,acoustic,0.572,0.454,3,-10.286,1,0.0258,0.4770,0.000014,0.0974,0.515,140.182,216387,4
2,Joshua Hyslop,Do Not Let Me Go,7BRCa8MPiyuvr2VU3O9W0F,57,2012,acoustic,0.409,0.234,3,-13.711,1,0.0323,0.3380,0.000050,0.0895,0.145,139.832,158960,4
3,Boyce Avenue,Fast Car,63wsZUhUZLlh1OsyrZq7sz,58,2012,acoustic,0.392,0.251,10,-9.845,1,0.0363,0.8070,0.000000,0.0797,0.508,204.961,304293,4
4,Andrew Belle,Sky's Still Blue,6nXIYClvJAfi6ujLiKqEq8,54,2012,acoustic,0.430,0.791,6,-5.419,0,0.0302,0.0726,0.019300,0.1100,0.217,171.864,244320,4


(2120676, 19)
artist_name              0
track_name               0
track_id                 0
popularity               0
year                     0
genre               960928
danceability             0
energy                   0
key                      0
loudness                 0
mode                     0
speechiness              0
acousticness             0
instrumentalness         0
liveness                 0
valence                  0
tempo                    0
duration_ms              0
time_signature           0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2120676 entries, 0 to 2120675
Data columns (total 19 columns):
 #   Column            Dtype  
---  ------            -----  
 0   artist_name       object 
 1   track_name        object 
 2   track_id          object 
 3   popularity        int64  
 4   year              int64  
 5   genre             object 
 6   danceability      float64
 7   energy            float64
 8   key               int64  
 9   l

### Removing duplicate songs

In [75]:
print(combined_dataset.shape)
combined_dataset = combined_dataset.drop_duplicates(subset=["track_name", "artist_name", "danceability", "energy", "key", "mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "time_signature"])
print(combined_dataset.shape)

(2120676, 19)
(2026473, 19)


In [79]:
from custom_utils import save_dataframe_as_parquet
save_dataframe_as_parquet(combined_dataset, folder_path="data", folder_name="preprocessed_spotify_data", always_overwrite=True)

Dataframe saved in 3 files under the folder: data/preprocessed_spotify_data


'data/preprocessed_spotify_data'